# Определение стоимости автомобилей

In [1]:
import pandas as pd

from lightgbm import LGBMRegressor
from sklearn.ensemble import RandomForestRegressor

from sklearn.preprocessing import OrdinalEncoder
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV

from sklearn.metrics import mean_squared_error

Сервис по продаже автомобилей с пробегом разрабатывает приложение для привлечения новых клиентов. В нём можно быстро узнать рыночную стоимость своего автомобиля. В вашем распоряжении исторические данные: технические характеристики, комплектации и цены автомобилей. Вам нужно построить модель для определения стоимости. 

Заказчику важны:

- качество предсказания (RMSE менее 2500);
- скорость предсказания;
- время обучения.

## Описание данных


### Признаки

- `DateCrawled` — дата скачивания анкеты из базы
- `VehicleType` — тип автомобильного кузова
- `RegistrationYear` — год регистрации автомобиля
- `Gearbox` — тип коробки передач
- `Power` — мощность (л. с.)
- `Model` — модель автомобиля
- `Kilometer` — пробег (км)
- `RegistrationMonth` — месяц регистрации автомобиля
- `FuelType` — тип топлива
- `Brand` — марка автомобиля
- `NotRepaired` — была машина в ремонте или нет
- `DateCreated` — дата создания анкеты
- `NumberOfPictures` — количество фотографий автомобиля
- `PostalCode` — почтовый индекс владельца анкеты (пользователя)
- `LastSeen` — дата последней активности пользователя


### Целевой признак

- `Price` — цена (евро)

## Подготовка данных

In [2]:
state = 12345

In [3]:
df_autos = pd.read_csv('/datasets/autos.csv')

In [4]:
df_autos.head(3)

,DateCrawled,Price,VehicleType,RegistrationYear,Gearbox,Power,Model,Kilometer,RegistrationMonth,FuelType,Brand,NotRepaired,DateCreated,NumberOfPictures,PostalCode,LastSeen
0,2016-03-24 11:52:17,480,NaN,1993,manual,0,golf,150000,0,petrol,volkswagen,NaN,2016-03-24 00:00:00,0,70435,2016-04-07 03:16:57
1,2016-03-24 10:58:45,18300,coupe,2011,manual,190,NaN,125000,5,gasoline,audi,yes,2016-03-24 00:00:00,0,66954,2016-04-07 01:46:50
2,2016-03-14 12:52:21,9800,suv,2004,auto,163,grand,125000,8,gasoline,jeep,NaN,2016-03-14 00:00:00,0,90480,2016-04-05 12:47:46


Приведем к нижнему регистру названия столбцов:

In [5]:
df_autos.columns.str.lower()

Index(['datecrawled', 'price', 'vehicletype', 'registrationyear', 'gearbox',
       'power', 'model', 'kilometer', 'registrationmonth', 'fueltype', 'brand',
       'notrepaired', 'datecreated', 'numberofpictures', 'postalcode',
       'lastseen'],
      dtype='object')

In [6]:
df_autos.columns = ['date_crawled', 'price', 'vehicle_type', 'registration_year', 'gear_box',
       'power', 'model', 'kilometer', 'registration_month', 'fuel_type', 'brand',
       'not_repaired', 'date_created', 'number_of_pictures', 'postal_code',
       'last_seen']

In [7]:
df_autos.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 354369 entries, 0 to 354368
Data columns (total 16 columns):
 #   Column              Non-Null Count   Dtype 
---  ------              --------------   ----- 
 0   date_crawled        354369 non-null  object
 1   price               354369 non-null  int64 
 2   vehicle_type        316879 non-null  object
 3   registration_year   354369 non-null  int64 
 4   gear_box            334536 non-null  object
 5   power               354369 non-null  int64 
 6   model               334664 non-null  object
 7   kilometer           354369 non-null  int64 
 8   registration_month  354369 non-null  int64 
 9   fuel_type           321474 non-null  object
 10  brand               354369 non-null  object
 11  not_repaired        283215 non-null  object
 12  date_created        354369 non-null  object
 13  number_of_pictures  354369 non-null  int64 
 14  postal_code         354369 non-null  int64 
 15  last_seen           354369 non-null  object
dtypes:

In [8]:
df_autos.isna().mean().sort_values(ascending=False)

not_repaired          0.200791
vehicle_type          0.105794
fuel_type             0.092827
gear_box              0.055967
model                 0.055606
date_crawled          0.000000
price                 0.000000
registration_year     0.000000
power                 0.000000
kilometer             0.000000
registration_month    0.000000
brand                 0.000000
date_created          0.000000
number_of_pictures    0.000000
postal_code           0.000000
last_seen             0.000000
dtype: float64

In [9]:
df_autos.corr()

,price,registration_year,power,kilometer,registration_month,number_of_pictures,postal_code
price,1.000000,0.026916,0.158872,-0.333199,0.110581,NaN,0.076055
registration_year,0.026916,1.000000,-0.000828,-0.053447,-0.011619,NaN,-0.003459
power,0.158872,-0.000828,1.000000,0.024002,0.043380,NaN,0.021665
kilometer,-0.333199,-0.053447,0.024002,1.000000,0.009571,NaN,-0.007698
registration_month,0.110581,-0.011619,0.043380,0.009571,1.000000,NaN,0.013995
number_of_pictures,NaN,NaN,NaN,NaN,NaN,NaN,NaN
postal_code,0.076055,-0.003459,0.021665,-0.007698,0.013995,NaN,1.000000


In [10]:
df_autos.describe()

,price,registration_year,power,kilometer,registration_month,number_of_pictures,postal_code
count,354369.000000,354369.000000,354369.000000,354369.000000,354369.000000,354369.0,354369.000000
mean,4416.656776,2004.234448,110.094337,128211.172535,5.714645,0.0,50508.689087
std,4514.158514,90.227958,189.850405,37905.341530,3.726421,0.0,25783.096248
min,0.000000,1000.000000,0.000000,5000.000000,0.000000,0.0,1067.000000
25%,1050.000000,1999.000000,69.000000,125000.000000,3.000000,0.0,30165.000000
50%,2700.000000,2003.000000,105.000000,150000.000000,6.000000,0.0,49413.000000
75%,6400.000000,2008.000000,143.000000,150000.000000,9.000000,0.0,71083.000000
max,20000.000000,9999.000000,20000.000000,150000.000000,12.000000,0.0,99998.000000


Предварительно можем сделать **выводы**:

- присутствуют пропуски в столбцах: `not_repaired`, `vehicle_type`, `fuel_type`, `gear_box, model`;
- признаки с датой имеют некорректный тип данных;
- в данных есть как количественные, так и категориальные признаки;
- корреляция между количественными признаками не обнаружена;
- столбец `number_of_pictures` полностью заполнен нулями, можем его удалить.

Заполним пропуски в зависимости от бренда и модели автомобиля:

In [11]:
df_autos['model'] = df_autos['model'].fillna(
    df_autos.groupby('brand')['model'].transform('first'))

In [12]:
df_autos['vehicle_type'] = df_autos['vehicle_type'].fillna(
    df_autos.groupby(['brand', 'model'])['vehicle_type'].transform('first'))

In [13]:
df_autos['gear_box'] =  df_autos['gear_box'].fillna(
    df_autos.groupby(['brand', 'model'])['gear_box'].transform('first'))

In [14]:
df_autos['fuel_type'] =  df_autos['fuel_type'].fillna(
    df_autos.groupby(['brand', 'model'])['fuel_type'].transform('first'))

In [15]:
df_autos['not_repaired'] = df_autos['not_repaired'].fillna(
    df_autos.groupby(['registration_year', 'kilometer'])['not_repaired'].transform('first'))

Заполним оставшиеся пропуски:

In [16]:
df_autos[['not_repaired', 'vehicle_type', 'fuel_type', 'gear_box', 'model']] = df_autos[
         ['not_repaired', 'vehicle_type', 'fuel_type', 'gear_box', 'model']].fillna('unknown')

In [17]:
df_autos.isna().mean().sort_values(ascending=False)

date_crawled          0.0
price                 0.0
vehicle_type          0.0
registration_year     0.0
gear_box              0.0
power                 0.0
model                 0.0
kilometer             0.0
registration_month    0.0
fuel_type             0.0
brand                 0.0
not_repaired          0.0
date_created          0.0
number_of_pictures    0.0
postal_code           0.0
last_seen             0.0
dtype: float64

Приведем столбцы с датами к соответствующему типу:

In [18]:
df_autos['date_crawled'] = pd.to_datetime(df_autos['date_crawled'], format='%Y-%m-%d %H:%M:%S')
df_autos['date_created'] = pd.to_datetime(df_autos['date_created'], format='%Y-%m-%d %H:%M:%S')
df_autos['last_seen'] = pd.to_datetime(df_autos['last_seen'], format='%Y-%m-%d %H:%M:%S')

In [19]:
df_autos[['date_crawled', 'date_created', 'last_seen']].head(3)

,date_crawled,date_created,last_seen
0,2016-03-24 11:52:17,2016-03-24,2016-04-07 03:16:57
1,2016-03-24 10:58:45,2016-03-24,2016-04-07 01:46:50
2,2016-03-14 12:52:21,2016-03-14,2016-04-05 12:47:46


In [20]:
(pd.DatetimeIndex(df_autos['date_crawled']).year).value_counts()

2016    354369
Name: date_crawled, dtype: int64

In [21]:
(pd.DatetimeIndex(df_autos['last_seen']).year).value_counts()

2016    354369
Name: last_seen, dtype: int64

In [22]:
(pd.DatetimeIndex(df_autos['date_created']).year).value_counts()

2016    354343
2015        25
2014         1
Name: date_created, dtype: int64

Создадим столбец с информацией о годе только из `date_created`, т.к. в остальных столбцах год одинаковый для всех объектов.

In [23]:
df_autos['date_created_year'] = pd.DatetimeIndex(df_autos['date_created']).year

Подготовим признаки:

In [24]:
features = df_autos.drop([
    'price', 'number_of_pictures', 'date_crawled', 'date_created', 'last_seen'], axis=1)

target = df_autos['price']

In [25]:
features.head(3)

,vehicle_type,registration_year,gear_box,power,model,kilometer,registration_month,fuel_type,brand,not_repaired,postal_code,date_created_year
0,small,1993,manual,0,golf,150000,0,petrol,volkswagen,no,70435,2016
1,coupe,2011,manual,190,a8,125000,5,gasoline,audi,yes,66954,2016
2,suv,2004,auto,163,grand,125000,8,gasoline,jeep,no,90480,2016


In [26]:
features_train, features_valid_test, target_train, target_valid_test = train_test_split(
    features, target, test_size = 0.4, random_state=state)

features_valid, features_test, target_valid, target_test = train_test_split(
    features_valid_test, target_valid_test, test_size = 0.5, random_state=state)

In [27]:
features_train.head(3)

,vehicle_type,registration_year,gear_box,power,model,kilometer,registration_month,fuel_type,brand,not_repaired,postal_code,date_created_year
51358,sedan,2006,manual,163,3er,150000,1,gasoline,bmw,no,40227,2016
259924,sedan,2012,manual,170,a3,50000,11,gasoline,audi,yes,92363,2016
144289,bus,2018,manual,109,berlingo,150000,11,petrol,citroen,no,32278,2016


Преобразуем признаки техникой Ordinal Encoding:

In [28]:
ctg = ['vehicle_type', 'gear_box', 'model', 'fuel_type', 'brand', 'not_repaired']

encoder = OrdinalEncoder()
encoder.fit(features_train[ctg])

features_train[ctg] = encoder.transform(features_train[ctg])
features_valid[ctg] = encoder.transform(features_valid[ctg])
features_test[ctg] = encoder.transform(features_test[ctg])

/tmp/ipykernel_189/1343842346.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  features_train[ctg] = encoder.transform(features_train[ctg])
/opt/conda/lib/python3.9/site-packages/pandas/core/indexing.py:1738: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value[:, i].tolist(), pi)
/tmp/ipykernel_189/1343842346.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the ca

In [29]:
features_train.head(3)

,vehicle_type,registration_year,gear_box,power,model,kilometer,registration_month,fuel_type,brand,not_repaired,postal_code,date_created_year
51358,4.0,2006,1.0,163,11.0,150000,1,2.0,2.0,0.0,40227,2016
259924,4.0,2012,1.0,170,28.0,50000,11,2.0,1.0,2.0,92363,2016
144289,0.0,2018,1.0,109,50.0,150000,11,6.0,5.0,0.0,32278,2016


## Обучение моделей

Обучим разные модели и подберем гиперпараметры:

### Модель Случайный лес

In [30]:
rfr = RandomForestRegressor(random_state=state)

parametrs = {'n_estimators': range (50, 51, 2),
              'max_depth': range (23, 24, 2)}

In [31]:
grid = GridSearchCV(rfr, parametrs, cv=3)
grid.fit(features_train[:1001], target_train[:1001])

grid.best_params_

{'max_depth': 23, 'n_estimators': 50}

In [32]:
model_forest = RandomForestRegressor(max_depth=23 , n_estimators=50 , random_state=state)

### Модель LightGBM

In [33]:
lgbm = LGBMRegressor(random_state=state)

parametrs = {'n_estimators': range (70, 71, 10),
              'max_depth': range (9, 10, 2),
              'num_leaves': range (29, 30, 2)}

In [34]:
grid = GridSearchCV(lgbm, parametrs, cv=3)
grid.fit(features_train[:1001], target_train[:1001])

grid.best_params_

{'max_depth': 9, 'n_estimators': 70, 'num_leaves': 29}

In [35]:
model_lgbm = LGBMRegressor(num_leaves=29, max_depth=9 , n_estimators=70 , random_state=state)

## Анализ моделей

Проанализируем скорость работы и качество моделей:

### Модель Случайный лес

In [36]:
%%time

model_forest.fit(features_train, target_train)

CPU times: user 40 s, sys: 430 ms, total: 40.5 s
Wall time: 40.5 s


RandomForestRegressor(max_depth=23, n_estimators=50, random_state=12345)

In [37]:
%%time

predictions_forest = model_forest.predict(features_valid)

CPU times: user 1.36 s, sys: 27.8 ms, total: 1.39 s
Wall time: 1.39 s


In [38]:
print('RMSE:', mean_squared_error(target_valid, predictions_forest)**0.5)

RMSE: 1768.720084646382


### Модель LightGBM

In [39]:
%%time

model_lgbm.fit(features_train, target_train)

CPU times: user 4.79 s, sys: 32.8 ms, total: 4.82 s
Wall time: 4.84 s


LGBMRegressor(max_depth=9, n_estimators=70, num_leaves=29, random_state=12345)

In [40]:
%%time

predictions_lgbm = model_lgbm.predict(features_valid)

CPU times: user 403 ms, sys: 3.57 ms, total: 407 ms
Wall time: 403 ms


In [41]:
print('RMSE:', mean_squared_error(target_valid, predictions_lgbm)**0.5)

RMSE: 1919.8233860187552


- Модель Случайный лес работает медленнее, чем LightGBM, особенно на этапе обучения.
- Значение метрики RMSE не превышает 2500 у обеих моделей, но наилучший результат показывает модель Случайный лес: 1769, следовательно, делаем выбор в пользу данной модели.

## Тестируем модель

In [42]:
%%time

predictions_test = model_forest.predict(features_test)

CPU times: user 1.37 s, sys: 7.81 ms, total: 1.37 s
Wall time: 1.4 s


In [43]:
print('RMSE:', mean_squared_error(target_test, predictions_test)**0.5)

RMSE: 1773.9596513643376


Результат получился близким к результату на валидационной выборке, следовательно, переобучения нет. Метрика RMSE равна 1774, что заметно меньше 2500 и нам подходит.